# C Interpreter

# Third Stage

Design a grammar using an a upward translator that accepts the following inputs from the C language:
- `printf` functions from C language.

Add the AST for arithmetic, logic and relation operators.

To develop this we will desing the grammar that accepts `printf` statements and then implement the AST for arithmetic, logic and relation operators.

## 1. Grammar design: Printf statement

We will start developing the *printf* statement:

### C print statement (printf)

In *C* the base structure of the print statement is:

``` 
printf("string [%type]*", [variable_name])
```

Could contains variables or not, and the numbers of tags must match the number of variables.

- printexpr <- `PRINTF` `(` stringexpr printftail
- printftrail <- `,` fact printftail
- printftrail <- `)`  
- stringexpr <- CSTRING
- fact -> `-` fact
- fact -> num
- fact -> `ID`

## Implementing the lexical analyzer (Lexer)

We will reuse the main lexical analyzer implementation, adding the *printf* logic necessary to process the new rules

In [35]:
from sly import Lexer

class Scanner(Lexer):
    tokens = {ID, CNUM, PRINTF, STYPE, CSTRING}
    literals ={'(', ')',',',';'}

    # Ignore whitespace and tabulations

    ignore = ' \t'

    # Regular expressions rules for tokens

    ID = r'[a-zA-Z][\w_]*'
    CSTRING = r'\"(\\.|[^\"])*\"'

    # Special cases
    ID['printf'] = PRINTF

    @_(r'\d+')
    def CNUM(self, t):
        t.value = int(t.value)
        return t

    # Error handling rule

    def error(self, t):
        print('<-'*10,"Illegal character '{}'".format(t.value[0]), '->'*10)
        self.index += 1
        t.type='Illegal'
        t.value =t.value[0]
        return t

## 1. Testing lexical Analyzer (printf)

In [33]:
import pandas as pd 

data = pd.read_csv('../assets/testing/printf_sentences.csv', delimiter="'")
data[['printf_sentences']]

,printf_sentences
0,"printf(""%d %d %d %d"", a, b, c, d);"
1,"printf(""%d %f"", 8, 45);"
2,"printf(""the add is %d the sub is %f"", 8, 45);"


In [1]:
lexer = Scanner()
sentences = data['printf_sentences'].values
pass_or_not = []
all_token_pass = True

for index, sentence in enumerate(sentences):
    print('-' * 80,"{} Lexically Testing sentence: '{}'".format(index, sentence),'-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not

NameError: name 'Scanner' is not defined

## 2. Developing the AST

Per symbol with a semtantic associated (operations, control statements, ID...) we have to implement a Node. But we can refactor operations using  Abstract class, and generalizing by operator type (unary or binary)

In [37]:
class Node():
    def write(self):
        pass

class BinaryOpNode(Node):
    def __init__(self, op, p1, p2):
        self.operation = op
        self.pn1 = p1
        self.pn2 = p2
    
    def write(self):
        print("{} {} {} = {}".format(self.pn1, self.operation, self.pn2, eval(str(self.pn1) + self.operation + str(self.pn2))))

class UnaryOpNode(Node):
    def __init__(self, op, p1):
        self.operation = op
        self.pn1 = p1

    def write(self):
        print("{} {} = {}".format( self.operation, self.pn1, eval(self.operation+" "+str(self.pn1))))

In [39]:
sum_op = BinaryOpNode('+', 4 ,5)
sum_op.write()

not_op = UnaryOpNode('not', True)
not_op.write()

neg_op = UnaryOpNode('-', 4)
neg_op.write()

4 + 5 = 9
not True = False
- 4 = -4
